In [1]:
!nvidia-smi

Fri Aug 16 09:41:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070        Off | 00000000:01:00.0  On |                  N/A |
|  0%   36C    P8               5W / 185W |    264MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [22]:
"""
Resnet-18 for classifying roof materials from PlanetScope SuperDove imagery
Case study in Washington, D.C. 
"""

import os, time, glob
import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import xarray as xr
import numpy as np
import rasterio as rio
import matplotlib.pyplot as plt
import gc
import torch
import torch.nn as nn
import optuna
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate
from torchvision import transforms, utils
from torchsat.models.classification import resnet18

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.utils import resample

from fiona.crs import from_epsg
from shapely.geometry import box

import warnings
warnings.filterwarnings("ignore")

plt.ion() # interactive

# Projection information
wgs = from_epsg(4326)
proj = from_epsg(32618)
print(f'Projected CRS: {proj}')

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping'

print("Successfully imported all packages!")

ModuleNotFoundError: No module named 'optuna'

In [3]:
class RoofImageDataset_Planet(Dataset):
    """Class to handle PlanetScope SuperDove imagery for Resnet-18"""

    def __init__(self, gdf, img_path, n_bands=8, imgdim=64, transform=None):
        """
        Args:
            gdf: Geodataframe containing 'geometry' column and 'class_code' column
            img_path: the path to the PlanetScope SuperDove composite image (single mosaic file)
                - see 'psscene-prep.py' for spectral indices calculation
            imgdim (int): Image dimension for CNN implementation
            transform (callable, optional): Optional transform to be applied on a sample

        Returns image chunks with class labels
        """

        if not os.path.exists(img_path):
            raise ValueError(f'Image does not exists: {img_path}')

        self.geometries = [p.centroid for p in gdf.geometry.values]
        self.img = img_path
        self.image_dim = imgdim # resnet window dimension, defaults to 64
        self.n_bands = n_bands
        self.Y = gdf.code.values
        # Define transforms
        if transform is not None:
            self.transform = transforms.Compose([
                transforms.Resize((self.image_dim, self.image_dim)),  # Resize to NxN for ResNet-18
                transforms.ToTensor()
            ])
        else:
            self.transform = transform
    
    def __len__(self):
        return len(self.geometries)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Sample the PlanetScope image
        left, bottom, right, top = self.geometries[idx].bounds
        geom = self.geometries[idx]

        try:
            sample = self.sample_image(geom)  # run the sampling function
            
            # if self.transform:
            #     sample = self.transform(sample)  # transform to common size

            cc = self.Y[idx]  # get the class codes
            
            # Ensure the sample has the correct dimensions
            if sample.shape[1:] != (self.image_dim, self.image_dim):
                raise ValueError(f'Invalid sample shape: {sample.shape}')
                        
        except Exception as e:
            print(f"Skipping invalid sample at index {idx}: {e}")
            return None  # Return None for invalid samples
        
        # Convert the sample array to a Torch object
        sample = torch.from_numpy(sample)

        return {'image': sample.type(torch.FloatTensor),
                'code': torch.tensor(cc).type(torch.LongTensor)}
        
        
    def sample_image(self, geom):
        # Sample the image at each geometry
        samples = [] # store the samples in a list

        N = self.image_dim
        half_N = self.image_dim / 2
            
        # Use the windows.from_bounds() method to return the window
        # Returns image chunks from training data locations
        with rio.open(self.img) as src:
            py, px = src.index(geom.x, geom.y)
            window = rio.windows.Window(px - N // 2, py - N // 2, N, N)
            
            # Read the data in the window
            # clip is a nbands * N * N numpy array
            clip = src.read(window=window)

            # Handle the case where the sample is smaller than the expected size
            if clip.shape != (self.n_bands, N, N):
                raise ValueError(f'Invalid sample shape: {clip.shape}\nAttempting to create padding ...')
                padding = [(0, 0), (0, max(0, N - clip.shape[1])), (0, max(0, N - clip.shape[2]))]
                clip = np.pad(clip, padding, mode='constant', constant_values=0)
            else:
                samples.append(clip)

            del clip, py, px, window

        # Convert the image chunk to a numpy array
        samples_arr = np.array(samples)

        del samples # Clear up memory

        # Make sure there is valid data
        if samples_arr.sum() > 0:
            ans = np.ma.masked_equal(samples_arr, 0).mean(axis=0)
        else:
            ans = samples_arr.mean(axis=0)

        del samples_arr

        return ans


def make_good_batch(batch):
    """
    Removes bad samples if image dimensions do not match.
    Args:
        - batch: list of dictionaries, each containing 'image' tensor and 'code' tensor
    returns: list of dictionaries same as input with samples having non-matching image dims removed
    """
    valid_samples = []
    for sample in batch:
        if sample is not None:
            image, code = sample['image'], sample['code']
            if code != 255 and not torch.isnan(code) and not torch.isinf(code):
                if not torch.isnan(image).any() and not torch.isinf(image).any():
                    if image.shape == (8, 64, 64):  # Ensure dimensions match
                        valid_samples.append(sample)
            del image, code

    if not valid_samples:
        return None

    return default_collate(valid_samples)
    del valid_samples, sample


def balance_sampling(df, ratio=1, strategy='undersample'):
    """
    Generate balanced sample from training data based on the given ratio.
    """
    # Get the class counts
    class_counts = df['class_code'].value_counts()
    min_class_count = class_counts.min()
    # Calculate the target count for each class based on the ratio
    target_count = {class_label: min(min_class_count * ratio, len(df[df['class_code'] == class_label]))
                    for class_label in class_counts.index}
    # Create an empty list to store balanced dataframes
    balanced_dfs = []
    for class_label in class_counts.index:
        class_df = df[df['class_code'] == class_label]
        if strategy == 'undersample':
            # Under-sample the majority class
            balanced_class_df = resample(
                class_df, replace=False, n_samples=target_count[class_label], random_state=42)
        elif strategy == 'oversample':
            # Over-sample the minority class
            balanced_class_df = resample(
                class_df, replace=True, n_samples=target_count[class_label], random_state=42)
        balanced_dfs.append(balanced_class_df)

    balanced_df = pd.concat(balanced_dfs)
    return balanced_df


print("Class and functions ready to use!")

Class and functions ready to use!


In [4]:
os.chdir('/home/jovyan')
print(os.getcwd())

/home/jovyan


In [5]:
# Load the training data (footprints)
ref_path = os.path.join('opp/data/dc_data_reference_footprints.gpkg')
ref = gpd.read_file(ref_path)
ref.head()

,class_code,areaUTMsqft,uid,description,code,geometry
0,CS,357.783709,1CS,Composition Shingle,0,"POLYGON ((324215.868 4313568.665, 324215.792 4..."
1,CS,918.640862,2CS,Composition Shingle,0,"POLYGON ((324602.816 4311717.247, 324604.322 4..."
2,CS,1383.414170,3CS,Composition Shingle,0,"POLYGON ((327253.581 4300371.859, 327258.154 4..."
3,CS,836.410297,4CS,Composition Shingle,0,"POLYGON ((333608.13 4306267.691, 333607.957 43..."
4,CS,330.514264,5CS,Composition Shingle,0,"POLYGON ((326482.699 4300939.466, 326487.386 4..."


In [6]:
# Calculate the 'optimal' window size from the footprint areas
mean_area_sqm = int(ref.areaUTMsqft.values.mean()) * 0.092903
pct90_area_sqm = np.percentile(ref.areaUTMsqft, 90) * 0.092903
print(f'Mean footprint area (sqm): {mean_area_sqm}')
print(f'90th percentile footprint area (sqm): {pct90_area_sqm}')
# Calculate the side length ('optimal' window size) * 3 
print(f'90th percentile side length (m): {int(np.sqrt(pct90_area_sqm))}')
window_size = (int(np.sqrt(pct90_area_sqm) * 3) - 1)
print(f'Potential optimal window size: {window_size}')

Mean footprint area (sqm): 99.499113
90th percentile footprint area (sqm): 158.13312473978425
90th percentile side length (m): 12
Potential optimal window size: 36


In [7]:
# Observe the class imbalance in the reference data
print(f"Class counts:\n\n{ref.class_code.value_counts()}\n")

Class counts:

class_code
ME    29651
CS    27687
SL    11080
UR     1018
WS      866
TL      617
SH      589
Name: count, dtype: int64



In [8]:
# Merge the shingle classes (wood shingle and shingle)
merge = {'WS': 'WSH', 'SH': 'WSH'}
ref['class_code'].replace(merge, inplace=True)
ref['code'], _ = pd.factorize(ref['class_code']) # create a factorized version
print(ref['class_code'].value_counts())  # check the counts

class_code
ME     29651
CS     27687
SL     11080
WSH     1455
UR      1018
TL       617
Name: count, dtype: int64


In [9]:
# Create a dictionary mapping class_code to code
class_mapping = dict(zip(ref['class_code'], ref['code']))
print(class_mapping)

{'CS': 0, 'ME': 1, 'SL': 2, 'UR': 3, 'TL': 4, 'WSH': 5}


In [10]:
# Perform balanced sampling (random undersampling)
ref_bal = balance_sampling(ref, ratio=10, strategy='undersample')
ref_bal.code.value_counts()

code
1    6170
0    6170
2    6170
5    1455
3    1018
4     617
Name: count, dtype: int64

In [11]:
# Split into train/test for each class
train_df, test_df, val_df = [], [], []

# Define split ratio
vs = 0.2  # Validation size ratio (20%)

# Perform stratified split to separate training data from validation data
train_df, val_df = train_test_split(
    ref_bal, # your filtered samples
    test_size=vs, 
    random_state=27, 
    stratify=ref_bal['code']
)

# Print the class distribution in training and validation sets to verify stratification
print("Train class distribution:\n", train_df['code'].value_counts())
print("Validation class distribution:\n", val_df['code'].value_counts())

Train class distribution:
 code
0    4936
1    4936
2    4936
5    1164
3     814
4     494
Name: count, dtype: int64
Validation class distribution:
 code
2    1234
1    1234
0    1234
5     291
3     204
4     123
Name: count, dtype: int64


In [12]:
# Load our image data to check on the format
stack_da_fp = os.path.join('opp/data/dc_data_psscene15b_norm_r.tif')
stack_da = rxr.open_rasterio(stack_da_fp, mask=True, cache=False).squeeze()
print(
    f"shape: {stack_da.rio.shape}\n"
    f"resolution: {stack_da.rio.resolution()}\n"
    f"bounds: {stack_da.rio.bounds()}\n"
    f"sum: {stack_da.sum().item()}\n"
    f"CRS: {stack_da.rio.crs}\n"
    f"NoData: {stack_da.rio.nodata}\n"
    f"Array: {stack_da}"
)
del stack_da

shape: (7555, 6046)
resolution: (3.0, -3.0)
bounds: (316269.0, 4295631.0, 334407.0, 4318296.0)
sum: 4.670212268829346
CRS: EPSG:32618
NoData: None
Array: <xarray.DataArray (band: 8, y: 7555, x: 6046)> Size: 1GB
[365420240 values with dtype=float32]
Coordinates:
  * band         (band) int64 64B 1 2 3 4 5 6 7 8
  * x            (x) float64 48kB 3.163e+05 3.163e+05 ... 3.344e+05 3.344e+05
  * y            (y) float64 60kB 4.318e+06 4.318e+06 ... 4.296e+06 4.296e+06
    spatial_ref  int64 8B 0
Attributes:
    TIFFTAG_IMAGEDESCRIPTION:  {"atmospheric_correction": {"aerosol_model": "...
    TIFFTAG_DATETIME:          2022:06:05 14:56:31
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        15203
    STATISTICS_MEAN:           662.44066810447
    STATISTICS_MINIMUM:        80
    STATISTICS_STDDEV:         475.12461911944
    STATISTICS_VALID_PERCENT:  42.96
    AREA_OR_POINT:             Area
    scale_factor:              1.0
    add_offset:                0.0
    long_name:   

In [13]:
# Set up the Resnet-18 model

# Define whether to leverage cpu or gpu (for my local machine it is only cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # get device for gpu or cpu
print(f'Using {device} for model dev ...')

# Grab the number of classes
n_classes = ref_bal.class_code.unique().shape[0]
print(f'There are {n_classes} roof type classes.')

# Define the Resnet-18 model (in_channels = number of bands in the image)
model = resnet18(n_classes, in_channels=8, pretrained=False)

# Make model parallel and on GPU
if torch.cuda.device_count() >= 1:
    print("Using ", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    model.to(device)
else:
    #ps_model = nn.DataParallel(ps_model)
    model = nn.DataParallel(model)
    print('Made cpu parallel')

Using cuda for model dev ...
There are 6 roof type classes.
Using  1 GPUs!


In [14]:
# Number of samples in each class
val_counts = list(train_df['code'].value_counts())
print(val_counts)

total_samples = sum(val_counts)

# Calculate class weights
class_weights = [total_samples / count for count in val_counts]
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Print the calculated class weights for verification
print(f"Class weights: {class_weights}")

# Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

[4936, 4936, 4936, 1164, 814, 494]
Class weights: tensor([ 3.5008,  3.5008,  3.5008, 14.8454, 21.2285, 34.9798], device='cuda:0')


In [17]:
gc.collect()

536

In [18]:
def objective_fun(trial):
    """ Objective function for hyperparameter tuning """
    """
    Function for fine-tuning Resnet-18 model using 'optuna' Python package
    Args:
        - trial: Optuna trial
    """

    # Suggest hyperparameters to test
    batch_size = trial.suggest_int('batch_size', 16, 128)
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    momentum = trial.suggest_uniform('momentum', 0.8, 0.99)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-1)

    # Load the train, test, and validation
    train_loader = DataLoader(train_df, batch_size=batch_size, shuffle=True, collate_fn=make_good_batch)
    val_loader = DataLoader(val_df, batch_size=batch_size, shuffle=False, collate_fn=make_good_batch)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    # Training loop
    model.train()
    for epoch in range(10):  # Adjust number of epochs as needed
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

In [19]:
# Create an Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Display the best hyperparameters and accuracy
print("Best hyperparameters:", study.best_params)
print("Best accuracy:", study.best_value)

NameError: name 'optuna' is not defined

In [29]:
gc.collect()

1750

In [ ]:
t0 = time.time()

# Create an Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective_fun, n_trials=50)

t1 = (time.time() - t0) / 60
print(f"Total elapsed time: {t1:.2f} minutes.")

In [ ]:
# Display the best hyperparameters and accuracy
print("Best hyperparameters:", study.best_params)
print("Best accuracy:", study.best_value)

In [33]:
gc.collect()

'/home/jovyan'